In [6]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Cleaning & Imputation/Output')
cur_date = "041823"

library(readxl)
library(tidyverse)
library(reshape2)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_041423.xlsx", sheet = 2))

In [7]:
head(mRNA_df)

,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
5,6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
6,1,F_1,ALDH1A3,C,F,1,72,-0.2119125,4.847400


In [8]:
# identifying the number of replicates for each sample condition
count_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Duplicate_Count = n()) %>% 
    arrange(-Duplicate_Count) 

head(count_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Duplicate_Count
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


In [9]:
replicate_count_df = count_mRNA_df %>%
    group_by(Duplicate_Count) %>%
    count() %>%
    arrange(-Duplicate_Count) 

replicate_count_df

Duplicate_Count,n
<int>,<int>
6,6
5,80
4,328
3,96
2,1340
1,2361


In [11]:
# creating a new column for replicate number
replicate_vector = c()
for (i in 1:length(replicate_count_df$n)){
    # calculating number of times to repeat the replicate number from 1 to the count
    replicate_count = replicate_count_df$n[i]/replicate_count_df$Duplicate_Count[i]
    replicate_iteration = rep(1:replicate_count_df$Duplicate_Count[i], times = replicate_count)
    
    # saving iteration
    replicate_vector = c(replicate_vector, replicate_iteration)
    
}

# adding to df 
count_mRNA_df$Replicate = replicate_vector
mRNA_df = count_mRNA_df %>%
    select(-Duplicate_Count)

head(mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Replicate
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,1
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,2
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,3
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,4
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,5
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


Removing variables

In [12]:
# only keeping variables with at least 25% of data 
variable_presence_df = mRNA_df %>% 
    # creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(ddCT_pslog2), 0, 1), all_count = 1) %>%
    group_by(mRNA, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    summarize(Variable_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
    arrange(Variable_Presence_Percent)

head(variable_presence_df)

`summarise()` has grouped output by 'mRNA', 'Condensate', 'Burn_Condition',
'Concentration'. You can override using the `.groups` argument.


mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Variable_Presence_Percent
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
ALDH1A3,C,F,1,24,100
ALDH1A3,C,F,1,72,100
ALDH1A3,C,F,25,4,100
ALDH1A3,C,F,25,24,100
ALDH1A3,C,F,25,72,100
ALDH1A3,C,F,5,24,100


In [ ]:
# # only keeping ids with at least 25% of data 
# variable_presence_df = mRNA_df %>% 
#     # creating 2 count cols: 1st for non-missing data, 2nd for all data
#     mutate(data_count = ifelse(is.na(ddCT_pslog2), 0, 1), all_count = 1) %>%
#     group_by(mRNA, Condensate, Burn_Condition, Concentration, Time_Point) %>%
#     summarize(Variable_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
#     arrange(Variable_Presence_Percent)

# head(variable_presence_df)

# QRILC Imputation